In [1]:
import numpy as np
import pandas as pd
import pyodbc
import warnings
import re
import nltk
from nltk import ngrams
from collections import defaultdict
import random
from itertools import islice
warnings.filterwarnings('ignore')

# from libraries.utils_predict import TextPredictor

In [2]:
conx_string = "DRIVER={ODBC Driver 17 for SQL Server}; SERVER=APCKRMPTMD01PV,41433; Database=Namlos; UID=Namlos_user; PWD=M@!!eL@498#;"
conx = pyodbc.connect(conx_string)
query = '''SELECT [Issue]
  FROM [Namlos].[dbo].[KB_View_Issue_PowerBI]'''


df = pd.read_sql(query, conx)
print(df.shape)
df.head()

(13561, 1)


,Issue
0,PPE diletakkan diluar kotak PPE. Ada barang2 l...
1,tali tas rawan nyangkut/kepleset
2,tray terlalu dekat dengan paha operator
3,Alat elektronik dekat kaki operator
4,Alat elektronik dekat kaki


In [3]:
df.dropna(subset=['Issue'], inplace=True)

In [43]:
def text_clean(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^A-Za-z\s]','', text)
    text = re.sub(r'\b(\w+)\.{3}', r'\1', text)

   
    # Convert to lowercase
    text = text.lower()

    # Remove Number 
    text = re.sub('[0-9]+', '', text)

    # Replace Word 
    text = text.replace('5 S', '5s').replace('5S', '5s').replace('c','')
    text = text.replace('improer','improper').replace('majunpanel','maju panel')

    # Remove 2 of word repetitif
    pattern = r'\b(\w+)(2)\b'
    text = re.sub(pattern, lambda match: f'{match.group(1)}', text)

    text = text.strip()

    # Separating conjoined words
    word_combine = ['nyangkut','kepleset','tray','kotak','ppe','tali','operator','alat','lantai','part','terjepit','sop','sesuai','kabel',
                    'stagging','meja','menjuntai','penempatan','mesin','development','panel','line','area','depan','kaki','logiform']
    text = re.sub(r'(' + '|'.join(word_combine) + r')', r'\1 ', text)
   
    return text

df['Issue'] = df['Issue'].apply(text_clean)

In [44]:
data = ''.join(df['Issue'])

In [45]:
words = nltk.word_tokenize(data)
words = [word.lower() for word in words if word.isalnum()]

words

['ppe',
 'diletakkan',
 'diluar',
 'kotak',
 'ppe',
 'ada',
 'barang',
 'lain',
 'yg',
 'tak',
 'terpakai',
 'diatas',
 'kotak',
 'ppe',
 'tali',
 'tas',
 'rawan',
 'nyangkut',
 'kepleset',
 'tray',
 'terlalu',
 'dekat',
 'dengan',
 'paha',
 'operator',
 'alat',
 'elektronik',
 'dekat',
 'kaki',
 'operator',
 'alat',
 'elektronik',
 'dekat',
 'kaki',
 'logiform',
 'hute',
 'terlepaskabel',
 'selang',
 'angin',
 'dekat',
 'kaki',
 'posisi',
 'lokasi',
 'dennison',
 'resleting',
 'terlalu',
 'mepet',
 'dari',
 'operator',
 'ke',
 'finished',
 'produtselang',
 'angin',
 'menjuntai',
 'pakaging',
 'tidak',
 'rapi',
 'di',
 'area',
 'supermarketblister',
 'ditaruh',
 'di',
 'rooted',
 'headoperator',
 'proses',
 'tag',
 'vinyl',
 'menggunakan',
 'anti',
 'fatigue',
 'matt',
 'double',
 'terhalang',
 'hute',
 'assortmentbeberapa',
 'line',
 'area',
 'spot',
 'welding',
 'kabel',
 'angin',
 'menjuntai',
 'kontainer',
 'box',
 'hairsetting',
 'diisi',
 'botol',
 'jeriken',
 'dan',
 'lain',
 'l

In [46]:
N = 4

ngrams_list = list(ngrams(words, N))

ngram_freq = defaultdict(int)
for ngram in ngrams_list:
    ngram_freq[ngram] += 1

In [47]:
def predict_next_words(prefix, num_predictions=1):
    prefix_len = len(prefix)
    matching_ngrams = [(ngram, freq) for ngram, freq in ngram_freq.items() if ngram[:prefix_len] == prefix]

    if not matching_ngrams:
        return ["No prediction available."]

    sorted_ngrams = sorted(matching_ngrams, key=lambda x: x[1], reverse=True)
    predictions = []
    used_words = set(prefix)

    for ngram, freq in sorted_ngrams:
        next_word = ngram[prefix_len]
        if next_word not in used_words:
            predictions.append(next_word)
            used_words.add(next_word)
        if len(predictions) >= num_predictions:
            break

    if not predictions:
        return ["No prediction available."]

    return predictions

In [48]:
user_input = input("Enter a prefix for next-word prediction: ").lower().split()
prefix = tuple(user_input)
prediction = predict_next_words(prefix,num_predictions=6)  

if prediction[0] == "No prediction available.":
    print("No predictions available for the given prefix.")
else:
    full_prediction = ' '.join(prefix) + ' ' + ' '.join(prediction)
    print(f"Prediction: {full_prediction}")

Prediction: elektronik dekat kaki
